# More power method (continuation of lecture 2)

In this continuation of lecture 2, we will see that having a good abstraction of hardware resources allows us to run the **same code** in parallel.

"Parallel computing will have made it when we never have to know any of the internal details." Alan Edelman

## Using parallel hardware

In [1]:
using LinearAlgebra

In [2]:
function power_method(M, v)
    T = eltype(v)
    for i in 1:100
        v = M*v        # repeatedly creates a new vector and destroys the old v
        v /= T(norm(v))
    end
    
    return v, T(norm(M*v)) / T(norm(v))  # or  (M*v) ./ v
end

power_method (generic function with 1 method)

First we create a standard Julia matrix (on the CPU):

In [ ]:
M = [2 1; 1 1.]

### DistributedArrays for large arrays spread across different processors

The Julia package [DistributedArrays.jl](https://github.com/JuliaParallel/DistributedArrays.jl) defines a `DArray` ("distributed array") type, which provides an abstraction that looks like a standard Julia array, but is spread across several different processors.

Since modern desktops and laptops often have multiple cores, we can use this.

First we allow Julia access to more processes:

In [ ]:
using Distributed

In [ ]:
addprocs(4)   # add cores to your Julia process

In [6]:
#]add DistributedArrays

In [7]:
using DistributedArrays

There are several ways to create `DArray`s:

In [8]:
M = drand(10, 10)

10×10 DArray{Float64,2,Array{Float64,2}}:
 0.797213  0.951436   0.656916   0.367544  …  0.903086  0.895331    0.348658
 0.339716  0.921422   0.799058   0.303904     0.158507  0.00050227  0.212506
 0.131629  0.0973171  0.0762106  0.373484     0.531555  0.314167    0.41875 
 0.511268  0.449071   0.601008   0.106522     0.762724  0.0451633   0.254405
 0.437012  0.922265   0.952716   0.480592     0.591622  0.391823    0.657018
 0.569394  0.195423   0.959914   0.488594  …  0.849583  0.99617     0.437544
 0.123654  0.280004   0.943045   0.536824     0.287063  0.915536    0.879087
 0.686117  0.471247   0.116463   0.520215     0.855908  0.518076    0.750019
 0.823241  0.159754   0.17866    0.637679     0.85278   0.112701    0.856088
 0.269538  0.757709   0.303293   0.126665     0.161356  0.317337    0.102364

If we really need to, we can find out where Julia is storing each piece of the array:

In [9]:
M.indices

2×2 Array{Tuple{UnitRange{Int64},UnitRange{Int64}},2}:
 (1:5, 1:5)   (1:5, 6:10) 
 (6:10, 1:5)  (6:10, 6:10)

This shows that the array was divided up into equal pieces on each of the four processors.

In [10]:
v = drand(10)

10-element DArray{Float64,1,Array{Float64,1}}:
 0.08812213640942379
 0.8799860870671785 
 0.6398420530320019 
 0.08135629150174095
 0.9598502246268485 
 0.845547943017942  
 0.11628654731689991
 0.4286055071471504 
 0.38237675695680995
 0.6548863175117976 

In [11]:
M * v

10-element DArray{Float64,1,Array{Float64,1}}:
 3.4594512387095055
 2.5708100111146392
 1.2863793868719497
 2.916879741387089 
 3.8433751978861492
 2.5272485636212085
 2.727578909940217 
 3.179995093148907 
 2.3599963371995325
 1.7310246056069172

Again, we see that `*` has been defined for these objects, so once again we can just run

In [12]:
power_method(M, v)

([0.419647, 0.2696, 0.177278, 0.305874, 0.369741, 0.327932, 0.275749, 0.414283, 0.324245, 0.170807], 4.918849245763999)

## Operators

Consider the following averaging operator that we could call a random walk or averaging operator:

In [3]:
averaging(n) = 0.5 * SymTridiagonal(zeros(n), ones(n-1))

averaging (generic function with 1 method)

In [4]:
averaging(7)

7×7 SymTridiagonal{Float64,Array{Float64,1}}:
 0.0  0.5   ⋅    ⋅    ⋅    ⋅    ⋅ 
 0.5  0.0  0.5   ⋅    ⋅    ⋅    ⋅ 
  ⋅   0.5  0.0  0.5   ⋅    ⋅    ⋅ 
  ⋅    ⋅   0.5  0.0  0.5   ⋅    ⋅ 
  ⋅    ⋅    ⋅   0.5  0.0  0.5   ⋅ 
  ⋅    ⋅    ⋅    ⋅   0.5  0.0  0.5
  ⋅    ⋅    ⋅    ⋅    ⋅   0.5  0.0

In [5]:
v = 1.0:2:13
averaging(7) * v

7-element Array{Float64,1}:
  1.5
  3.0
  5.0
  7.0
  9.0
 11.0
  5.5

In [16]:
averaging(7)

7×7 SymTridiagonal{Float64,Array{Float64,1}}:
 0.0  0.5   ⋅    ⋅    ⋅    ⋅    ⋅ 
 0.5  0.0  0.5   ⋅    ⋅    ⋅    ⋅ 
  ⋅   0.5  0.0  0.5   ⋅    ⋅    ⋅ 
  ⋅    ⋅   0.5  0.0  0.5   ⋅    ⋅ 
  ⋅    ⋅    ⋅   0.5  0.0  0.5   ⋅ 
  ⋅    ⋅    ⋅    ⋅   0.5  0.0  0.5
  ⋅    ⋅    ⋅    ⋅    ⋅   0.5  0.0

In [17]:
v

1.0:2.0:13.0

In [6]:
power_method(averaging(7), v)

([0.198757, 0.339299, 0.479841, 0.479841, 0.479841, 0.339299, 0.198757], 0.9238795325112868)

Although we have saved some memory by using a `SymTridiagonal` structure, we clearly are still storing far more information than we need to, since this is just "0 on the diagonal and 0.5 on the super- and sub-diagonal".

We can define a new type in Julia to reflect this. We realise that we do **not actually need to store any information inside the "matrix"**. In fact, we will rather define a **linear operator**, just as we would really like to do in mathematics:

In [1]:
struct AveragingOp
    # contains *no* information
end

We have a "dummy type" that contains no information. It will be interesting because of "what it can do", i.e. the operations that we define that involve objects of this type.

We create an object of this type, called `A`, with

In [2]:
A = AveragingOp()  # default constructor

AveragingOp()

In [3]:
A

AveragingOp()

We will define what it means to multiply an object of this type by a vector. The simplest case would be

In [4]:
import Base.*  # necessary to overload *

function *(A::AveragingOp, v::AbstractVector)
    v  # just the identity operator
end

* (generic function with 344 methods)

which gives an identity operator:

In [5]:
v = [1, 2, 43.0]
A*v

3-element Array{Float64,1}:
  1.0
  2.0
 43.0

In [31]:
power_method(A, v)

([0.0232244, 0.0464489, 0.998651], 1.0)

We now define the actual averaging operation. It takes a vector and returns a new vector:

In [6]:
function *(A::AveragingOp, v::AbstractVector)
    [ v[1];    # ; concatenates
      [(v[i-1] + v[i+1])/2  for i in 2:length(v)-1];    # array comprehension
      v[end] 
    ]
end

* (generic function with 344 methods)

In [7]:
v = (1:7).^2
@show v
A*v

v = [1, 4, 9, 16, 25, 36, 49]


7-element Array{Float64,1}:
  1.0
  5.0
 10.0
 17.0
 26.0
 37.0
 49.0

In [13]:
dump(A)

AveragingOp AveragingOp()


Since `*` now works, we can again just reuse our some generic `power_method` implementation:

In [14]:
power_method(A, float.(v))

([0.0127339, 0.114605, 0.216477, 0.318349, 0.42022, 0.522092, 0.623963], 1.0000000107136144)

You could worry that `*` is not the correct notation. Mathematically, for an operator $\mathcal{L}$ operating on a vector $\mathbf{v}$, we might write $\mathcal{L} \mathbf{v}$, just using juxtaposition. Unfortunately, we are unable to use this notation in Julia.

We could instead use a `⋅` for juxtaposition. Now that we have defined `*`, we can just do

In [38]:
import LinearAlgebra.⋅
A::AveragingOp ⋅ v = A * v

⋅ (generic function with 1 method)

In [39]:
A ⋅ v

7-element Array{Float64,1}:
  1.0
  5.0
 10.0
 17.0
 26.0
 37.0
 49.0

We can even define $\mathcal{L}(\mathbf{v})$:

In [8]:
(A::AveragingOp)(v) = A*v


In [9]:
v

7-element Array{Int64,1}:
  1
  4
  9
 16
 25
 36
 49

In [10]:
A(v)

7-element Array{Float64,1}:
  1.0
  5.0
 10.0
 17.0
 26.0
 37.0
 49.0

In [31]:
hcat([ A * ( ((1:7).==k) .+ 0) for k=1:7]...)

7×7 Array{Float64,2}:
 1.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.5  0.0  0.5  0.0  0.0  0.0  0.0
 0.0  0.5  0.0  0.5  0.0  0.0  0.0
 0.0  0.0  0.5  0.0  0.5  0.0  0.0
 0.0  0.0  0.0  0.5  0.0  0.5  0.0
 0.0  0.0  0.0  0.0  0.5  0.0  0.5
 0.0  0.0  0.0  0.0  0.0  0.0  1.0

In [17]:
identity(7)

7

In [47]:
Matrix{Int}(n,n)

UndefVarError: UndefVarError: n not defined

In [48]:
?Matrix

search: Matrix BitMatrix DenseMatrix StridedMatrix AbstractMatrix



```
Matrix{T} <: AbstractMatrix{T}
```

Two-dimensional dense array with elements of type `T`, often used to represent a mathematical matrix. Alias for [`Array{T,2}`](@ref).

---

```
Matrix{T}(undef, m, n)
```

Construct an uninitialized [`Matrix{T}`](@ref) of size `m`×`n`. See [`undef`](@ref).

# Examples

```julia-repl
julia> Matrix{Float64}(undef, 2, 3)
2×3 Array{Float64,2}:
 6.93517e-310  6.93517e-310  6.93517e-310
 6.93517e-310  6.93517e-310  1.29396e-320
```

---

```
Matrix{T}(nothing, m, n)
```

Construct a [`Matrix{T}`](@ref) of size `m`×`n`, initialized with [`nothing`](@ref) entries. Element type `T` must be able to hold these values, i.e. `Nothing <: T`.

# Examples

```jldoctest
julia> Matrix{Union{Nothing, String}}(nothing, 2, 3)
2×3 Array{Union{Nothing, String},2}:
 nothing  nothing  nothing
 nothing  nothing  nothing
```

---

```
Matrix{T}(missing, m, n)
```

Construct a [`Matrix{T}`](@ref) of size `m`×`n`, initialized with [`missing`](@ref) entries. Element type `T` must be able to hold these values, i.e. `Missing <: T`.

# Examples

```jldoctest
julia> Matrix{Union{Missing, String}}(missing, 2, 3)
2×3 Array{Union{Missing, String},2}:
 missing  missing  missing
 missing  missing  missing
```


In [49]:
T = Tridiagonal(rand(5), rand(6), rand(5))

6×6 Tridiagonal{Float64,Array{Float64,1}}:
 0.283719     0.576758   ⋅         ⋅         ⋅         ⋅      
 0.000569716  0.753393  0.976017   ⋅         ⋅         ⋅      
  ⋅           0.433551  0.295944  0.240744   ⋅         ⋅      
  ⋅            ⋅        0.21205   0.554782  0.178021   ⋅      
  ⋅            ⋅         ⋅        0.233174  0.368639  0.79962 
  ⋅            ⋅         ⋅         ⋅        0.313972  0.454543

In [50]:
dump(T)

Tridiagonal{Float64,Array{Float64,1}}
  dl: Array{Float64}((5,)) [0.000569716, 0.433551, 0.21205, 0.233174, 0.313972]
  d: Array{Float64}((6,)) [0.283719, 0.753393, 0.295944, 0.554782, 0.368639, 0.454543]
  du: Array{Float64}((5,)) [0.576758, 0.976017, 0.240744, 0.178021, 0.79962]
  du2: #undef


In [51]:
@which T*rand(6)

*(A::AbstractArray{T,2}, x::AbstractArray{S,1}) where {T, S} in LinearAlgebra at /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.0/LinearAlgebra/src/matmul.jl:49

In [12]:
(x::Int64)(y::Int64) = x+y

In [15]:
five = 5
five(6)

11

In [16]:
5(6)

30

In [18]:
Base.:*(n::Number,::typeof(!)) = factorial(n)

In [19]:
5 * !

120

In [26]:
Base.:*(n::Number,f::Function) = f(n)

In [35]:
3(!)

6

In [33]:
3*sin*cos*√

0.9950171283751177

In [32]:
√(cos(sin(3)))

0.9950171283751177

In [36]:
x=10
5x

50

In [37]:
5*x

50

In [38]:
x = rand(3,3)

3×3 Array{Float64,2}:
 0.220525  0.482147  0.97731 
 0.847238  0.431971  0.970275
 0.90841   0.946529  0.464383

In [39]:
1000x

3×3 Array{Float64,2}:
 220.525  482.147  977.31 
 847.238  431.971  970.275
 908.41   946.529  464.383

In [2]:
4(3)

12

In [4]:
x=3
4(x) ## This is simply *(4,x)

12

In [5]:
4(!) ## could maybe mean *(4,!) or 4 * ! , if only ! had a meaning

MethodError: MethodError: no method matching *(::Int64, ::typeof(!))
Closest candidates are:
  *(::Any, ::Any, !Matched::Any, !Matched::Any...) at operators.jl:502
  *(::T<:Union{Int128, Int16, Int32, Int64, Int8, UInt128, UInt16, UInt32, UInt64, UInt8}, !Matched::T<:Union{Int128, Int16, Int32, Int64, Int8, UInt128, UInt16, UInt32, UInt64, UInt8}) where T<:Union{Int128, Int16, Int32, Int64, Int8, UInt128, UInt16, UInt32, UInt64, UInt8} at int.jl:54
  *(::Union{Int16, Int32, Int64, Int8}, !Matched::BigInt) at gmp.jl:463
  ...

In [6]:
Base.:*(n::Int64, ::typeof(!)) = factorial(n)

In [8]:
*(4,!)

24

In [9]:
4*!

24

In [10]:
4(!)

24

In [11]:
4!

LoadError: syntax: extra token "!" after end of expression

In [12]:
# using LinearAlgebra
exp(rand(3,3))

3×3 Array{Float64,2}:
 1.76167  0.149753  0.21952 
 1.32114  1.96301   0.338723
 1.23581  1.05462   1.41627 

In [13]:
rand(3,3)^0

3×3 Array{Float64,2}:
 1.0  0.0  0.0
 0.0  1.0  0.0
 0.0  0.0  1.0

In [14]:
using LinearAlgebra

In [15]:
I

UniformScaling{Bool}
true*I

In [17]:
ones(Int, 3,3) + 100I

3×3 Array{Int64,2}:
 101    1    1
   1  101    1
   1    1  101

In [19]:
Matrix{Int64}(I,3,3)

3×3 Array{Int64,2}:
 1  0  0
 0  1  0
 0  0  1

In [20]:
I(n) = Matrix{Int64}(I,n,n)

ErrorException: error in method definition: function LinearAlgebra.I must be explicitly imported to be extended

In [21]:
typeof(I)

UniformScaling{Bool}

In [22]:
(::UniformScaling)(n::Int64)  = Matrix{Int64}(I,n,n)

In [23]:
I(4)

4×4 Array{Int64,2}:
 1  0  0  0
 0  1  0  0
 0  0  1  0
 0  0  0  1

In [32]:
UniformScaling{Int64}(100)

UniformScaling{Int64}
100*I

In [34]:
UniformScaling{Bool}(true)

UniformScaling{Bool}
true*I

In [36]:
UniformScaling{Bool}(false) * rand(7,7)

7×7 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [37]:
56I

UniformScaling{Int64}
56*I